In [25]:
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import clear_output
from qiskit.circuit.library import RealAmplitudes, TwoLocal
from qiskit_algorithms.optimizers import COBYLA, L_BFGS_B
from qiskit_machine_learning.algorithms.classifiers import NeuralNetworkClassifier, VQC
from qiskit_machine_learning.utils.loss_functions.loss_functions import Loss
from sklearn.datasets import load_iris
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
from sklearn.model_selection import train_test_split
import os
from term_loss.loss import L1Loss_ERM, L1Loss_TERM, L2Loss_ERM, L2Loss_TERM, CrossEntropyLoss_ERM, CrossEntropyLoss_TERM

In [88]:
iris_data = load_iris()
features = iris_data.data
labels = iris_data.target
features = MinMaxScaler(feature_range=(-np.pi, np.pi)).fit_transform(features)
split = 5
excess_class = 2
train_index_total = pd.read_excel(f"data/train_index_split{split}.xlsx", header=None).to_numpy()
test_index_total = pd.read_excel(f"data/test_index_split{split}.xlsx", header=None).to_numpy()
train_index = train_index_total[:, excess_class]
test_index = test_index_total[:, excess_class]
train_features = features[train_index]
train_labels = labels[train_index]
test_features = features[test_index]
test_labels = labels[test_index]

In [89]:
#Automated testing
num_points = 10
total_train = np.zeros(num_points)
total_test = np.zeros(num_points)
ansatz_reps_list = [1,2,3]

for j in ansatz_reps_list:
    for i in range(num_points):
        num_qubits = features.shape[1]
        feature_reps = 1
        ansatz_reps = j
        maxiter = 100
        t = 1
        feature_map = RealAmplitudes(num_qubits=num_qubits, entanglement="linear", parameter_prefix='x', reps=feature_reps, skip_final_rotation_layer=True)
        ansatz = RealAmplitudes(num_qubits=num_qubits, reps=ansatz_reps, entanglement='linear', skip_final_rotation_layer=False)
        initial_point = (np.random.random(ansatz.num_parameters) - 0.5) * 2 * np.pi 

        # construct neural network classifier
        vqc = VQC(
            feature_map=feature_map,
            ansatz=ansatz,
            optimizer=L_BFGS_B(maxiter=maxiter),
            loss = L2Loss_TERM(t=t),
            initial_point = initial_point
        )
        erm_string = f'term_t{t}'

        newpath = f'/Users/choyboy/Documents/Python/TERM/vqc_data/lbfgs/l2_loss/{erm_string}/split{split}_class{excess_class}/a{ansatz_reps}'
        if not os.path.exists(newpath):
            os.makedirs(newpath)

        vqc.fit(train_features, train_labels)
        train_score = vqc.score(train_features, train_labels)
        test_score = vqc.score(test_features, test_labels)
        total_train[i] = train_score
        total_test[i] = test_score
        print('Iteration', i+1, 'Train Score:', train_score, 'Test Score:', test_score)
        vqc.save(f'{newpath}/point{i+1}')

    # save total_train and total_test into excel files
    mean_train = np.mean(total_train)
    mean_test = np.mean(total_test)
    print('Mean Train Score:', mean_train, 'Mean Test Score:', mean_test)

    df_results = pd.DataFrame({'Train Accuracy': total_train, 'Test Accuracy': total_test})
    df_results.to_excel(f'{newpath}/results.xlsx', index=False)

Iteration 1 Train Score: 0.9107142857142857 Test Score: 0.9285714285714286
Iteration 2 Train Score: 0.8571428571428571 Test Score: 0.7857142857142857
Iteration 3 Train Score: 0.9107142857142857 Test Score: 0.9285714285714286
Iteration 4 Train Score: 0.8571428571428571 Test Score: 0.7857142857142857
Iteration 5 Train Score: 0.8571428571428571 Test Score: 0.7857142857142857
Iteration 6 Train Score: 0.8392857142857143 Test Score: 0.7857142857142857
Iteration 7 Train Score: 0.9285714285714286 Test Score: 0.7142857142857143
Iteration 8 Train Score: 0.9107142857142857 Test Score: 0.9285714285714286
Iteration 9 Train Score: 0.8035714285714286 Test Score: 0.7857142857142857
Iteration 10 Train Score: 0.8928571428571429 Test Score: 0.9285714285714286
Mean Train Score: 0.8767857142857143 Mean Test Score: 0.8357142857142857
Iteration 1 Train Score: 0.9285714285714286 Test Score: 0.9285714285714286
Iteration 2 Train Score: 0.9285714285714286 Test Score: 0.8571428571428571
Iteration 3 Train Score: 0

In [14]:
mean_train = np.mean(total_train)
mean_test = np.mean(total_test)
print('Mean Train Score:', mean_train, 'Mean Test Score:', mean_test)

Mean Train Score: 0.9524999999999999 Mean Test Score: 0.9499999999999998
